Problem https://projecteuler.net/problem=948

# Main idea

L will win the game for a word $\omega$ if the word $\omega$ has a suffix $\omega'$ where the number of L is $\ge$ of the number of R in $\omega'$ (except for the case when the word is $LR$, because at first we need to eliminate one letter).

The same occurs with R winnig the game, but with prefix instead of suffix.

So with that in mind, we can define a word with 4 attributes: 
* **Left_wins**
* **Right_wins**
* **number_L_from_R**: max number of the difference of the L and R from a suffix of $\omega$
* **number_R_from_L**: difference of R and L of the full word $\omega$

Every word with the same state $S$ will generate a word of state $S1$ if we add another letter at the right.

Note that if we have a word $\omega_R$ where R wins if start, R will win with any word that has $\omega_R$ as prefix, because R can remove all the right letters of $\omega_R$ in the first turn.

So, we have two cases wehn adding a letter to the right:
* If we add an L to the right:
  * the **number_L_from_R** will increase if its a non negative number, else will be 1 (because we will have a word with a suffix L).
  * the **number_R_from_L** will decrease by 1
  * **Left_wins** will be True, because **number_L_from_R** will be at least 1
  * **Right_wins** if right wins in the previous state or if the **number_R_from_L** is -1 (because R can remove the last L, think of LR -> LRL).
* If we add an R to the right:
  * the **number_L_from_R** will decrease by 1
  * the **number_R_from_L** will increase by 1
  * **Left_wins** will be True if **number_L_from_R** $\ge 0$ except for the case when the size of the word is 2
  * **Right_wins** if right wins in the previous state or if the **number_R_from_L** is greater than 0

The first state will be of words of len 1 (L and R).

We can generate the states of the words of size N+1 states adding an L and an R to all the states of the words of size N.


In [1]:
from collections import Counter

In [2]:
def update_states(states, new_letter: str, l_word: int):
    win_L = states[0]
    win_R = states[1]
    number_L_from_R = states[2]
    number_R_from_L = states[3]

    if new_letter == 'L':
        if number_L_from_R >= 0:
            number_L_from_R += 1
        else:
            number_L_from_R = 1
        number_R_from_L -= 1
        win_L = True
        win_R = win_R or number_R_from_L == -1
    else:
        number_R_from_L += 1
        number_L_from_R -= 1
        win_R = win_R or number_R_from_L > 0
        win_L = number_L_from_R >= 0 if l_word > 2 else number_L_from_R > 0
    new_states = (win_L, win_R, number_L_from_R, number_R_from_L)
    return new_states

In [3]:
words_states = {
    (True, False, 1, -1): 1,  # L
    (False, True, -1, 1): 1,  # R
}
for l in range(2, 61):
    new_word_states = Counter()
    for states, count in words_states.items():
        new_word_states[update_states(states, 'L', l)] += count
        new_word_states[update_states(states, 'R', l)] += count
    words_states = new_word_states.copy()

sum(count for states, count in words_states.items() if states[0] and states[1])

CPU times: total: 46.9 ms
Wall time: 42.1 ms


1033654680825334184